In [ ]:
# jupyter-notebook V0_1_1.ipynb --port 8888 --ip 192.168.1.5

import cv2
import os
import numpy as np
from collections import Counter
import itertools  
from collections import defaultdict
import time
import numba as nb
import copy
from os import listdir, mkdir
from os.path import isfile, isdir, join

from components_container import ComponentsContainer

%matplotlib inline

In [45]:
@nb.jit()
def change_color_of_indices(markers, indices_to_change, color):
    for i in range(markers.shape[0]):
        for j in range(markers.shape[1]):
            if markers[i,j] in indices_to_change:
                markers[i,j] = color
    return markers

@nb.jit()
def edges_close(edge1x, edge1y, edge2x, edge2y, thr):
    thrs = thr ** 2
    for i in range(len(edge1x)):
        x1 = edge1x[i]
        y1 = edge1y[i]
        for j in range(len(edge2x)):
            x2 = edge2x[j]
            y2 = edge2y[j]
            delta1 = x1 - x2
            delta2 = y1 - y2
                       
            if (delta1 ** 2 + delta2**2) < thrs:
                return True, ((x1, y1), (x2, y2))
    return False, None


class ComponentsContainer():
    
    def __init__(self, binary, low):
        
        nbComponents, markers, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8)

        components_index = {}
        background_size = max(stats[:, -1])
        small_comp_indices = []
        for i in range(len(stats)):
            if low < stats[i, -1] < background_size:
                components_index[i] = Component(stats[i, -1], centroids[i], i)
            else:
                small_comp_indices.append(i)
        markers = change_color_of_indices(markers, small_comp_indices, 0)

        self.binary = binary
        self.markers = markers
        self.components_index = components_index
        self.nucleus_labs = list()
        self.axon_labs = list()

        self.add_edge_info_to_components()


    
    def add_edge_info_to_components(self):
        for i in range(1, self.markers.shape[0] - 1):
            for j in range(1, self.markers.shape[1] - 1):
                cur_lab = self.markers[i, j]
                if cur_lab in self.components_index.keys() and \
                        not cur_lab == \
                                self.markers[i, j + 1] == \
                                self.markers[i, j - 1] == \
                                self.markers[i + 1, j] == \
                                self.markers[i - 1, j]:
                    self.components_index[cur_lab].add_edge(np.array([i, j]))

    @nb.jit()
    def merge_components_closer_than(self, centroid_t, contour_t):
        pairs_to_merge = []
        for first, second in itertools.combinations(self.components_index.keys(), 2):
            close_flag, points = self.components_index.get(first).is_close(self.components_index.get(second),
                                                                           centroid_t, contour_t)
            if close_flag:
                pairs_to_merge.append((first, second, points))
                
        starttime = time.time()
        for tm in pairs_to_merge:
            self._merge_two_components(*tm)


    def split_nucl_axon(self, threshold):
        self.nucleus_labs = list()
        self.axon_labs = list()
        
        for lab in self.components_index.keys():
            if not self.components_index.get(lab).label == lab:
                continue
            if self.components_index.get(lab).size > threshold:
                self.components_index[lab] = Nucleus.from_component(self.components_index[lab])
                self.nucleus_labs.append(lab)
            else:
                self.components_index[lab] = Axon.from_component(self.components_index[lab])
                self.axon_labs.append(lab)
        
    def group_axons_to_nucleus(self, centroid_t, contour_t):

        axons_with_nucl = 0
        for nucl_lab in self.nucleus_labs:
            nucl = self.components_index.get(nucl_lab)
            for axon_lab in self.axon_labs:
                axon = self.components_index.get(axon_lab)
                close, point = nucl.is_close(axon, centroid_t, contour_t)
                if (not axon.attached) and close:
                    nucl.axons.append(axon)
                    axon.attached = True
                    axons_with_nucl += 1


    def _merge_two_components(self, survivor_label, disappearing_label, joint_points):
        disappearing_label = self.get_correct_label(disappearing_label)
        survivor_label = self.get_correct_label(survivor_label)
        if survivor_label == disappearing_label:
            return

        self.components_index[survivor_label].size += self.components_index[disappearing_label].size
        self.components_index[survivor_label].edge.extend(self.components_index[disappearing_label].edge)
        self.components_index[disappearing_label].label = self.components_index[survivor_label].label

        self.markers[self.markers == disappearing_label] = survivor_label

        cv2.line(self.markers, (joint_points[0][1], joint_points[0][0]),
                 (joint_points[1][1], joint_points[1][0]), survivor_label, lineType=4,
                 thickness=2)

    def get_correct_label(self, label):
        if label == self.components_index.get(label).label:
            return label
        else:
            return self.get_correct_label(self.components_index.get(label).label)



class Component:
    def __init__(self, size, centroid, label):
        self.size = size
        self.centroid = centroid
        self.label = label
        self.edge = []

    @classmethod
    def from_component(cls, component):
        return cls(component.size, component.centroid, component.label, component.edge)

    def add_edge(self, point):
        self.edge.append(point)

    def is_close(self, second_comp, centroid_t, contour_t):
        if self.is_centroid_close(second_comp, centroid_t):
            return self.is_contour_close(second_comp, contour_t)
        else:
            return False, None

    def is_centroid_close(self, second_comp, thr):
        return np.linalg.norm(self.centroid - second_comp.centroid) < thr

    def is_contour_close(self, second_comp, thr):
        
        e1 = np.array(self.edge)
        e2 = np.array(second_comp.edge)
        return edges_close(e1[:,0],e1[:,1], e2[:,0],e2[:,1], thr)
    
    def is_nucleus(self):
        return False


class Nucleus(Component):
    def __str__(self):
        
        return "nucl size: {}, border_len: {}, axon_count: {}".format(self.size, len(self.edge), len(self.axons))
    def __init__(self, size, centroid, label, edge, axons=list()):
        super().__init__(size, centroid, label)
        if axons:
            self.axons = axons
        else:
            self.axons = list()
            
        self.edge = edge


    def is_nucleus(self):
        return True


class Axon(Component):
    def __str__(self):
        
        return "axon -- size: {}, border_len: {}".format(self.size, len(self.edge))
    def __init__(self, size, centroid, label, edge, attached=False):
        super().__init__(size, centroid, label)
        self.attached = attached
        self.edge = edge

In [2]:
# filter all noise using minimum size threshold
BACKGROUND_INTENSITY_THRESHOLD = 20
MIN_AXON_SIZE = 20

# merging close components based on centroids and contour distance
PREMERGE_CENTROIDS_DISTANCE_T  = 200
PREMERGE_CONTOUR_DISTANCE_T = 15

#splitting nucleus and axons based on size
MIN_NUCLEUS_SIZE = 150

#grouping possible axons to nucleus
CENTROIDS_DISTANCE_T = 90
CONTOUR_DISTANCE_T = 25

LOADPATH = '../raw_pics'
SAVEPATH = '../info'
RECURSIVE = True

In [6]:
def mirror_dir_for_save(loadpath):
    try:
        mkdir(loadpath.replace(LOADPATH, SAVEPATH))
    except(OSError):
        pass
    
def process_dir(loadpath):
    mirror_dir_for_save(loadpath)
    object_names_in_folder = listdir(loadpath)
    print ("start processing folder {} pics: {}".format(loadpath, str(len(object_names_in_folder))))
    for f in object_names_in_folder:
        local_object = join(loadpath, f)

        if isfile(local_object) and f[0] != '.':
            process_pic(join(loadpath, f))
            continue

        if RECURSIVE and isdir(join(loadpath, f)):
            process_dir(join(loadpath, f))
            continue

def process_pic(path):
    starttime = time.time()
    print("processing pic {} ...".format(path), end = "")
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, BACKGROUND_INTENSITY_THRESHOLD, 400, cv2.THRESH_BINARY)
    container = ComponentsContainer(binary, MIN_AXON_SIZE)
    container.merge_components_closer_than(PREMERGE_CENTROIDS_DISTANCE_T, PREMERGE_CONTOUR_DISTANCE_T)
    container.split_nucl_axon(MIN_NUCLEUS_SIZE)
    container.group_axons_to_nucleus(CENTROIDS_DISTANCE_T, CONTOUR_DISTANCE_T)
    with open(path.replace(LOADPATH, SAVEPATH).replace(".bmp", ".txt"), "w") as f:
        write_info_from_container(container, f)
        
    print('done in {} sec.'.format(time.time() - starttime))


def write_info_from_container(container, f):
    labs = sorted(container.nucleus_labs, key = lambda x: -container.components_index.get(x).size)
    f.write("#, Size, Perimeter, P/S\n")
    i = 0
    for nucl_lab in labs:
        i +=1
        nucl = container.components_index.get(nucl_lab)
        f.write("{:2}, {:4}, {:4}, {:.3}\n".format(i, nucl.size, len(nucl.edge), len(nucl.edge)/nucl.size))

In [7]:
process_dir(LOADPATH)

start processing folder ../raw_pics pics: 6
start processing folder ../raw_pics/.ipynb_checkpoints pics: 0
start processing folder ../raw_pics/day4 pics: 3
start processing folder ../raw_pics/day4/day4 pics: 5
start processing folder ../raw_pics/day4/day4/c4 pics: 7
processing pic ../raw_pics/day4/day4/c4/c4-ba-cortex_1z.bmp ...done in 11.66797685623169 sec.
processing pic ../raw_pics/day4/day4/c4/c4-ba-cortex_2z.bmp ...done in 6.745084762573242 sec.
processing pic ../raw_pics/day4/day4/c4/c4-ba-cortex_3z.bmp ...done in 11.628473043441772 sec.
processing pic ../raw_pics/day4/day4/c4/c4-ba-cortex_4z.bmp ...done in 9.22665286064148 sec.
processing pic ../raw_pics/day4/day4/c4/c4-ba-cortex_left_1z.bmp ...

KeyboardInterrupt: 

In [ ]:
# to do
# Write function to compute distance between contours. 

# more ideas:
# search for axons form axons
# implement contour filtering of axons
# add intensity related features
# Check if two centroids is closely contcted and merge them if needed.